In [2]:
#class for node
class Node():
    def __init__(self,state,parent,action):
        self.state = state
        self.parent = parent
        self.action = action

In [ ]:
#class for the StackFrontier


class StackFrontier():
    def __init__(self):
        self.frontier= []
    
    def add(self,node):
        self.frontier.append(node)
    
    def contains_node(self,state):
        return any(node.state == state for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0
    
    def remove(self):
        if  self.empty():
            return Exception("no soution for empty frontier")
        else:
            return self.frontier.pop() #pop is enough 